# 1 - Objetivo

A intenção deste exercício é observar como se comportam os algoritmos baseados em informação. Neste primeiro momento nós vamos utilizar funções padrão da biblioteca scikit-learn para estudar estes algoritmos  de machine learning. 

A [documentação do scikit-learn](https://scikit-learn.org/stable/modules/classes.html) é abrangente e possui diversos links com a parte teórica dos algoritmos, tutoriais e exemplos práticos.

## 2 - Carregando as bibliotecas

Scikit-learn possui uma interface limpa e intuitiva e todos os componentes da biblioteca expõem a mesma interface de métodos. Sendo uma biblioteca fácil de usar, tornou-se padrão na indústria de tecnologia.

Para utilizá-la, vamos primeiro carregar os métodos/módulos necessários, além de outras bibliotecas necessárias para o tutorial.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["figure.figsize"] = [6.4, 4.8]

import graphviz 
import pandas as pd
import seaborn as sns
sns.set()

from yellowbrick.style import set_palette
set_palette('yellowbrick')

## 3 - Preparação do dataset

Vamos começar a preparação do `dataset` carregando-o com a biblioteca pandas

In [6]:
data_file = 'grad_admission.csv'

In [7]:
dataset = pd.read_csv(data_file)

Vamos olhar alguns exemplos do `dataset` para enterndermos sua estrutura:

In [8]:
dataset.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research,Admit
0,337,118,4,4.5,4.5,9.65,Yes,0.92
1,324,107,4,4.0,4.5,8.87,Yes,0.76
2,316,104,3,3.0,3.5,8.00,Yes,0.72
3,322,110,3,3.5,2.5,8.67,Yes,0.80
4,314,103,2,2.0,3.0,8.21,No,0.65


Para facilitar a execução do resto do código, vamos separar os nomes das colunas em `features` e `target`.

In [9]:
columns = dataset.columns
columns

Index(['GRE', 'TOEFL', 'Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Admit'], dtype='object')

In [10]:
# aqui selecionamos os valores da lista, com exceção do último 
# para obter os nomes das features
features = columns[0:-1]
features

Index(['GRE', 'TOEFL', 'Rating', 'SOP', 'LOR', 'CGPA', 'Research'], dtype='object')

In [11]:
# aqui selecionamos o último valor da lista para identificar o target
target = 'y'
target

'y'

Utilizaremos este dataset mas criaremos um target artificial: se Admit for maior que 0.7, consideramos que o candidato será aceito (true); caso contrário não será aceito ( false). Desta forma, teremos um problema de classificação.

In [12]:
grads = dataset.copy()
grads['Admit'] = (grads['Admit'] > 0.7).astype('str')
grads = grads.rename(columns={'Admit':'y'})

In [13]:
grads.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research,y
0,337,118,4,4.5,4.5,9.65,Yes,True
1,324,107,4,4.0,4.5,8.87,Yes,True
2,316,104,3,3.0,3.5,8.00,Yes,True
3,322,110,3,3.5,2.5,8.67,Yes,True
4,314,103,2,2.0,3.0,8.21,No,False


### 3.1 - Transformação dos Dados

A implementação das árvores de decisão na biblioteca `scikit-learn` somente aceita a utilização de `features` numéricas. Por isso, precisamos converter as `features` catergóricas em representações numéricas.

Para isso, utilizaremos uma classe disponível na biblioteca que faz a conversão das categorias para formato numérico:

* `OrdinalEncoder` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder)

Para isso, nós: 

1. primeiro selecionaremos as colunas que precisam de conversão

2. criaremos um objeto `OrdinalEncoder`

3. faremos uma cópia do `dataset` original

4. transformaremos o `dataset`


*Observação:* algumas vezes, o `dataset`pode consumir muita memória RAM e, nestes casos, deve-se transformar a versão original do `dataset` e não criando a cópia.




In [14]:
# selecionando as colunas
columns_to_tranform = ['Research']

In [15]:
# importando o OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder

In [16]:
# criando o objeto e treinando-o no dataset
encoder = OrdinalEncoder()
encoder.fit(grads[columns_to_tranform])

OrdinalEncoder()

In [17]:
# criando a cópia do dataset
transformed_dataset = grads.copy()

In [18]:
# transformação dos dados
transformed_dataset[columns_to_tranform] = encoder.transform(grads[columns_to_tranform])

Para verificar se tudo está OK, vamos inspecionar algumas linhas do `dataset` transformado.

In [19]:
transformed_dataset.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research,y
0,337,118,4,4.5,4.5,9.65,1.0,True
1,324,107,4,4.0,4.5,8.87,1.0,True
2,316,104,3,3.0,3.5,8.00,1.0,True
3,322,110,3,3.5,2.5,8.67,1.0,True
4,314,103,2,2.0,3.0,8.21,0.0,False


### 3.2 - Dividindo o `dataset` em treino e teste

Uma das premissas do machine learning é que você não utilize os mesmos dados de treino para testar e avaliar o modelo. Ainda não vimos este assunto nas aulas (iniciaremos avaliação de modelos na 5a aula - semana que vem), mas por enquanto iremos dividir o `dataset` mesmo assim.

O scikit-learn possui uma função muito útil para nos auxiliar: `train_test_split`.

A assinatura do método com os valores `default` ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split)) é:

```{python}
x_train, x_test, y_train, y_test = train_test_split(*arrays, test_size=None, 
    train_size=None, random_state=None, suffle=True, stratify=None)
```

O retorno do método traz uma sequência ordenada de `numpy array` (ou `dataframe`, dependendo do formato dos `datasets` passados): `features` dos exemplos de treino; `features` dos exemplos de teste; `targets` dos exemplos de treino; `targets` dos exemplos de teste.

Por convenção, muitos praticantes de machine learning utilizam a letra `X` (ou `x`) para indicar a parte que contém as `features` e `Y` (ou `y`) para indicar a parte que contém os `targets`. Além disso é comum adicionar uma indicação do tipo de dataset que está sendo criado (`train`, `test` ou `valid`).

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(transformed_dataset[features], 
                                                    transformed_dataset[target], 
                                                    test_size=0.33,
                                                    random_state=42,
                                                    shuffle=True)

Vamos mais uma vez inspecionar os `datasets` de treino e de testes:

In [21]:
x_train.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
471,311,103,3,2.0,4.0,8.09,0.0
26,322,109,5,4.5,3.5,8.80,0.0
7,308,101,2,3.0,4.0,7.90,0.0
453,319,103,3,2.5,4.0,8.76,1.0
108,331,116,5,5.0,5.0,9.38,1.0


In [22]:
x_test.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
361,334,116,4,4.0,3.5,9.54,1.0
73,314,108,4,4.5,4.0,9.04,1.0
374,315,105,2,2.0,2.5,7.65,0.0
155,312,109,3,3.0,3.0,8.69,0.0
104,326,112,3,3.5,3.0,9.05,1.0


### 3.3 - Os modelos

O scikit-learn é uma biblioteca que implementa (quase) todos os principais algoritmos usados em machine learning, assim como diversos métodos utilitários para feature engineering e feature selection.

O principal método que todo o algoritmo implementa é o  ```fit```. Desta forma, fica fácil criarmos/substituirmos um algoritmo:

```
algoritmo.fit(features, alvos)
```

O scikit-learn implementa diversos parâmetros para cada algoritmo e, em geral, possui um valor default para todos eles. Mesmo assim, podemos alterar esses valores default usando o padrão python ```param=valor```:


```
algoritmo = Algoritmo(parametro1=valor1, parametro2=valor2, ...)
```

Cada algoritmo possui uma série de parâmetros que em geral diferem uns dos outros. Para verificar quais parâmetros são implementados por cada algoritmo e o significado de cada um, podemos consultar o link http://scikit-learn.org/stable/modules/classes.html

### 3.4 - Árvores de Decisão

Agora temos tudo em mãos para treinar a nossa primeira 
arvore de decisão utilizando `scikit-learn`.

As árvores de decisão estão implementadas dentro do submódulo `sklearn.tree` e são chamadas de `DecisionTreeClassifier` quando o `target` é uma classe ou `DecisionTreeRegressor` quando o `target` é um contínuo.

A assinaura da `DecisionTreeClassifier` com parâmetros `default` desta implementação é:

```{python}
DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, 
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
    max_features=None, random_state=None, max_leaf_nodes=None, 
    min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, 
    presort='deprecated', ccp_alpha=0.0)
```

Algumas observações:



*   Caso queiramos treinar um modelo com técnicas de `boosting`, basta setarmos o valor do parâmetro `class_weight` para  

```{python}
from sklearn.utils.class_weight import compute_class_weight
class_weights = class_weight.compute_class_weight(
    'balanced', np.unique(y_train), y_train) 
```                                       
Também é bastante útil quando temos um `dataset` "desbalanceado" (*i.e.*, quando uma classe possui muito mais exemplos que as outras).

*   Esta implementação já realiza `pre-pruning` por padrão quando setamos alguns ou todos os parâmetros: `min_samples_split, min_samples_leaf, min_weight_fraction_leaf,  max_features, max_leaf_nodes, min_impurity_decrease, min_impurity_split`. Dependendo da combinação destes parâmetros o `prepruning` também será diferente. 



Abaixo importamos o `DecisionTreeClassifier` para o nosso tutorial além de algumas outras funções auxiliares:

* `export_graphviz` que irá nos auxiliar na visualização da árvore gerada;

* e `classification_report` que nos auxiliará na visualização da performance(iremos ver esta função em mais detalhes no tutorial sobre avaliação de modelos).

In [23]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report

Antes de continuar, iremos criar uma função auxiliar para visualização das árvores treinadas:

In [24]:
def visualize_tree(model): 
    tree_data = export_graphviz(
        model0, 
        out_file=None, 
        feature_names=features,  
        class_names=transformed_dataset['y'].unique(),  
        filled=True, 
        proportion=True,
        rounded=True,  
        special_characters=True)  
    graph = graphviz.Source(tree_data)  
    return graph

Realizando o treino do modelo:

In [39]:
model0 = DecisionTreeClassifier(
    criterion='gini', 
    splitter='best', 
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features=7, 
    random_state=42, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    class_weight=None, 
    ccp_alpha=0.0)

model0.fit(x_train, y_train)

y_pred = model0.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.78      0.77      0.78        74
        True       0.82      0.82      0.82        91

    accuracy                           0.80       165
   macro avg       0.80      0.80      0.80       165
weighted avg       0.80      0.80      0.80       165



#### Visualizando a árvore

O `scikit-learn` possui algumas funções interessantes para visualizar as árvores treinadas. Para entender um pouco melhor, analise o código doa função `visualize_tree` definida um pouco mais acima ou então o código deste [link](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html#sklearn.tree.export_graphviz).

In [ ]:
visualize_tree(model0)

### 3.5 - *Ensembles*

O `scikit-learn` ainda possui diversas versões de árovres de decisão treinadas como *ensembles*, utilizando técnicas de `bagging` ou `boosting`. Todos estão implementados no submódulo `sklearn.ensemble`. Entre os principais, destacam-se:

* `RandomForestClassifier` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

* `AdaBoostClassifier` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)

* `BaggingClassifier` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

Como forma de avaliá-los, vamos treinar uma `RandomForest` com seus valores padrão:

In [36]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(
    n_estimators=100, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='auto', 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    #min_impurity_split=None, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=None, 
    random_state=None, 
    verbose=0,
    warm_start=False, 
    class_weight=None, 
    ccp_alpha=0.0, 
    max_samples=None)

rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.84      0.80      0.82        74
        True       0.84      0.88      0.86        91

    accuracy                           0.84       165
   macro avg       0.84      0.84      0.84       165
weighted avg       0.84      0.84      0.84       165



### 3.6 - xgboost

Há ainda uma outra forma de gerar `ensembles` combinando ambas as técnicas de `boosting` e `bagging` em um algoritmo bastante poderoso, o `xgboost` [docs](https://xgboost.readthedocs.io/en/latest/python/index.html). Este algoritmo é considerao *estado-da-arte*  para problemas de classificação utilizando `datasets` estruturados em `features`.

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    base_score=0.5, 
    booster='gbtree', 
    colsample_bylevel=1,
    colsample_bynode=1, 
    colsample_bytree=1, 
    gamma=0,
    learning_rate=0.1, 
    max_delta_step=0, 
    max_depth=3,
    min_child_weight=1, 
    missing=None, 
    n_estimators=100,
    n_jobs=1,
    nthread=None, 
    objective='binary:logistic', 
    random_state=0,
    reg_alpha=0, 
    reg_lambda=1, 
    scale_pos_weight=1, 
    seed=None,
    silent=None, 
    subsample=1, 
    verbosity=1
)

xgb_model.fit(x_train, y_train)

y_pred = xgb_model.predict(x_test)
print(classification_report(y_test, y_pred))